In [2]:
import pandas as pd
import numpy as np
import os
from numpy import random
import scipy 
from scipy import stats
from matplotlib import pyplot as plt
from scipy.stats import binom
from scipy.stats import poisson
from scipy.stats import norm ,t
import seaborn as sns
import pylab
from pylab import legend,plot,show,title,xlabel,ylabel
from random import choice
import statsmodels
from statsmodels import stats
from statsmodels.stats import weightstats as ssw
import statsmodels.api as sm
from statsmodels.formula.api import ols
import statsmodels.stats.multicomp
from statsmodels.stats.multicomp import pairwise_tukeyhsd
from statsmodels.stats import proportion as ssp
from scipy.stats import chisquare
from scipy.stats import chi2_contingency


from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

#### Now here , the response variable is categorical

Response can be nominal or ordinal.

Types:

1. Binary
2. Nominal (Multi categories, **without** logical order)
3. Ordinal (Multi categories, **with** logical order)

# 1] Binary Regression

- Response is either 0 or 1.

In [5]:
df_cars = pd.read_excel("CDAC_Databook.xlsx", sheet_name = 'mtcars')

In [6]:
df_cars.head()

,mpg,cyl,disp,hp,drat,wt,qsec,vs,am,gear,carb
0,21.0,6,160.0,110,3.90,2.620,16.46,0,1,4,4
1,21.0,6,160.0,110,3.90,2.875,17.02,0,1,4,4
2,22.8,4,108.0,93,3.85,2.320,18.61,1,1,4,1
3,21.4,6,258.0,110,3.08,3.215,19.44,1,0,3,1
4,18.7,8,360.0,175,3.15,3.440,17.02,0,0,3,2


In [7]:
df_cars = df_cars[['hp', 'wt', 'am']]

In [8]:
x_train = df_cars.drop('am', axis = 1)

In [9]:
y_train = df_cars['am']

In [10]:
x_train = sm.add_constant(x_train, prepend=False)

In [11]:
mod4 = sm.Logit(y_train, x_train).fit()

Optimization terminated successfully.
         Current function value: 0.157174
         Iterations 9


In [12]:
print(mod4.summary())

                           Logit Regression Results                           
Dep. Variable:                     am   No. Observations:                   32
Model:                          Logit   Df Residuals:                       29
Method:                           MLE   Df Model:                            2
Date:                Tue, 24 Dec 2024   Pseudo R-squ.:                  0.7673
Time:                        22:37:53   Log-Likelihood:                -5.0296
converged:                       True   LL-Null:                       -21.615
Covariance Type:            nonrobust   LLR p-value:                 6.267e-08
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
hp             0.0363      0.018      2.044      0.041       0.001       0.071
wt            -8.0835      3.069     -2.634      0.008     -14.098      -2.069
const         18.8663      7.444      2.535      0.0

- **Ho(hp)** : *hp* does not influence choice between 0 and 1 (manual or automatic)
- **Ho(wt)** : *wt* does not influence choice between 0 and 1 (manual or automatic)

#### Conclusion:

- *hp* influences choice between 0 and 1 (manual or automatic) (0.041 < 0.05)

1. wt has influences choice between 0 and 1 (manual or automatic) (0.008 < 0.05)
2. Probability of '1' will increase when you decrease the value of wt (They have negative correlation )

Odd's ratio = p / (1-p)
- Log of a value which is less than 1 would be negative

In [17]:
np.log(0.7/(1-0.7))

0.8472978603872034

In [18]:
np.log(0.3/(1-0.3))

-0.8472978603872036

In [19]:
x_train.head()

,hp,wt,const
0,110,2.620,1.0
1,110,2.875,1.0
2,93,2.320,1.0
3,110,3.215,1.0
4,175,3.440,1.0


In [20]:
mydata = pd.DataFrame([[90, 2.5, 1], [100, 2.5, 1], [110, 2.5, 1]], columns = x_train.columns)

In [21]:
mydata

,hp,wt,const
0,90,2.5,1
1,100,2.5,1
2,110,2.5,1


In [22]:
mod4.predict(mydata)

0    0.872207
1    0.907474
2    0.933747
dtype: float64

In [23]:
#np.log(0.907474/(1-0.907474)))-np.log(0.872207/(1-0.872207))))/10

In [24]:
(np.log(0.933747/(1-0.933747)) - np.log(0.907474/(1-0.907474)))/10

0.03625495472432383

In [25]:
mydata2 = pd.DataFrame([[90, 1, 1], [90, 2, 1], [90, 3, 1]], columns = x_train.columns)

In [26]:
mydata2

,hp,wt,const
0,90,1,1
1,90,2,1
2,90,3,1


In [27]:
mod4.predict(mydata2)

0    0.999999
1    0.997433
2    0.107061
dtype: float64

In [28]:
df_diab = pd.read_excel('CDAC_Databook.xlsx', sheet_name='diabetes')

In [29]:
df_diab.head()

,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,DietType,Outcome
0,148,72,35,0,33.6,0.63,50,1,1
1,85,66,29,0,26.6,0.35,31,2,0
2,183,64,0,0,23.3,0.67,32,1,1
3,89,66,23,94,28.1,0.17,21,1,0
4,137,40,35,168,43.1,2.29,33,1,1


In [30]:
df_diab = df_diab[['Glucose', 'DietType', 'Outcome']]

In [31]:
diet_dum = pd.get_dummies(df_diab['DietType'], drop_first=True).astype(int)
df_diab = df_diab.drop('DietType', axis = 1)
df_diab = pd.concat([df_diab, diet_dum], axis = 1)


In [32]:
df_diab.head()

,Glucose,Outcome,2,3
0,148,1,0,0
1,85,0,1,0
2,183,1,0,0
3,89,0,0,0
4,137,1,0,0


In [33]:
x_train, x_test, y_train, y_test = train_test_split(df_diab.drop('Outcome', axis = 1), df_diab['Outcome'], test_size=0.2, random_state=20)

In [34]:
x_train = sm.add_constant(x_train, prepend=False)

In [35]:
mod2 = sm.Logit(y_train, x_train).fit()

Optimization terminated successfully.
         Current function value: 0.342252
         Iterations 7


In [36]:
print(mod2.summary())

                           Logit Regression Results                           
Dep. Variable:                Outcome   No. Observations:                  614
Model:                          Logit   Df Residuals:                      610
Method:                           MLE   Df Model:                            3
Date:                Tue, 24 Dec 2024   Pseudo R-squ.:                  0.4715
Time:                        22:37:54   Log-Likelihood:                -210.14
converged:                       True   LL-Null:                       -397.59
Covariance Type:            nonrobust   LLR p-value:                 6.038e-81
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Glucose        0.0392      0.005      8.507      0.000       0.030       0.048
2             -3.8331      0.310    -12.357      0.000      -4.441      -3.225
3             -2.7269      0.328     -8.312      0.0

- Change in the odd's ratio when glucose is increased by 1 unit ---> m1

In [38]:
x_test = sm.add_constant(x_test)
y_pred = np.round(mod2.predict(x_test), 0)

C:\Users\Abhay\anaconda3\Lib\site-packages\statsmodels\discrete\discrete_model.py:2385: RuntimeWarning: overflow encountered in exp
  return 1/(1+np.exp(-X))


In [39]:
confusion_matrix(y_test, y_pred)

array([[101,   0],
       [ 53,   0]], dtype=int64)

In [40]:
confusion_matrix(y_test, y_pred)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.66      1.00      0.79       101
           1       0.00      0.00      0.00        53

    accuracy                           0.66       154
   macro avg       0.33      0.50      0.40       154
weighted avg       0.43      0.66      0.52       154



C:\Users\Abhay\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\Abhay\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\Abhay\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [41]:
89/98

0.9081632653061225

In [42]:
89/101


0.8811881188118812

In [43]:
89+12 # people who dont have diabetes

101

In [44]:
9+44 # people who have diabetes

53

## Nominal Logistic Regression

- A multinomial logistic regression (or multinomial regression for short) is used when the **outcome variable being predicted is nominal** and **has more than two categories that do not have a given rank or order**.
- This model can be used with any number of independent variables that are categorical or continuous.

In [85]:
df_nom = pd.read_excel('CDAC_DataBook.xlsx',sheet_name='nominal')

In [87]:
df_nom.head()

,ses,write,math,prog
0,1,35,41,1
1,2,33,41,2
2,3,39,44,3
3,1,37,42,1
4,2,31,40,2


In [89]:
np.unique(df_nom['prog'])

array([1, 2, 3], dtype=int64)

In [91]:
df_nom = df_nom.drop('write', axis = 1)

In [93]:
ses_dum = pd.get_dummies(df_nom['ses'], drop_first=True, prefix = 'season').astype(int)

In [95]:
df_nom = df_nom.drop('ses', axis = 1)

In [97]:
df_nom = pd.concat([df_nom, ses_dum], axis = 1)

In [99]:
df_nom.head()

,math,prog,season_2,season_3
0,41,1,0,0
1,41,2,1,0
2,44,3,0,1
3,42,1,0,0
4,40,2,1,0


In [101]:
x_train, x_test, y_train, y_test = train_test_split(df_nom.drop('prog', axis = 1), df_nom.prog, test_size=0.2, random_state = 20)

In [103]:
x_train = sm.add_constant(x_train, prepend=False)

In [110]:
x_train.head()

,math,season_2,season_3,const
152,51,1,0,1.0
67,49,1,0,1.0
133,57,0,1,1.0
59,54,0,1,1.0
63,40,1,0,1.0


In [112]:
y_train.head()

152    3
67     2
133    3
59     2
63     2
Name: prog, dtype: int64

The code provided below is performing a multinomial logistic regression using the `statsmodels` library in Python.

1. **`sm.MNLogit(y_train, x_train)`**: This part initializes a multinomial logistic regression model where:
   - `y_train`: This is the target variable (dependent variable) that contains the classes or categories we want to predict.
   - `x_train`: This is the input data (independent variables) used to predict the classes in `y_train`.

2. **`.fit()`**: This method fits the model to the provided training data. It estimates the parameters (coefficients) of the model based on the relationship between the independent variables in `x_train` and the dependent variable in `y_train`.

3. **`print(mod1.summary())`**: This line prints a summary of the fitted model, which includes information such as the coefficients for each predictor, standard errors, z-scores, p-values, and model statistics. The summary helps evaluate the model’s performance and the significance of the predictors.

Overall, this code is setting up a multinomial logistic regression model, fitting it to the training data, and then displaying a summary of the results.

In [105]:
mod1 = sm.MNLogit(y_train, x_train).fit()
print(mod1.summary())

Optimization terminated successfully.
         Current function value: 0.653667
         Iterations 8
                          MNLogit Regression Results                          
Dep. Variable:                   prog   No. Observations:                  160
Model:                        MNLogit   Df Residuals:                      152
Method:                           MLE   Df Model:                            6
Date:                Tue, 24 Dec 2024   Pseudo R-squ.:                  0.3687
Time:                        22:54:20   Log-Likelihood:                -104.59
converged:                       True   LL-Null:                       -165.68
Covariance Type:            nonrobust   LLR p-value:                 5.678e-24
    prog=2       coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
math           0.0112      0.036      0.312      0.755      -0.059       0.082
season_2       3.2174      0.

- prog 1 is refrence event
- ses 1 is refrence event
